In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
# Imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math

import preprocessing_utils as helper

%matplotlib inline

In [91]:
# Directory setup
BASE_DIR = '..'

TFRECORDS_DIR = '../data/eth_lsms_tfrecords_raw/'
LSMS_CSV_PATH = '../data/eth_lsms_clusters.csv'

EXPORT_DIR = '../data/eth_lsms_tfrecords/'

In [92]:
# avg_rad is the nightlights band
BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP', 'VIIRS', 'DELTA_TEMP', 'CO']

In [93]:
df_lsms = pd.read_csv(LSMS_CSV_PATH)

# Type conversion to float32
df_lsms['lat'] = df_lsms['lat'].astype(np.float32)
df_lsms['lon'] = df_lsms['lon'].astype(np.float32)
df_lsms['consumption'] = df_lsms['consumption'].astype(np.float32)
df_lsms = df_lsms.sort_values('consumption')

print(df_lsms.dtypes)
display(df_lsms)

country         object
consumption    float32
density          int64
lat            float32
lon            float32
dtype: object


country  consumption  density        lat        lon
0     ethiopia     0.061442        1  14.029943  38.541546
1     ethiopia     0.065432        1  14.029943  38.541546
2     ethiopia     0.112242        1   9.328209  42.142937
3     ethiopia     0.123466        1   7.879094  37.580292
4     ethiopia     0.214926        1   6.375622  42.246292
...        ...          ...      ...        ...        ...
6508  ethiopia    48.304890        0   8.968655  38.755150
6509  ethiopia    51.741539        0   8.978013  40.152702
6510  ethiopia    55.944542        0  13.490549  39.473690
6511  ethiopia    58.324226        0   8.987993  38.758446
6512  ethiopia    98.604965        0   9.346658  42.075893

[6513 rows x 5 columns]

In [100]:
# Searches for tfrecord by latitude and longitude
# tfrecords = sorted([f for f in os.listdir(EXPORT_DIR) if not f.startswith('.')])
# matched_record = ""

# for tf_r in tfrecords:
#     sample_tfrecord = EXPORT_DIR + tf_r
    
#     feature_set = helper.create_single_feature_set(filename=sample_tfrecord)
#     feature_array = helper.get_feature_array(feature_set=feature_set)
#     lat = np.float32(feature_set['lat'].float_list.value[0])
#     lon = np.float32(feature_set['lon'].float_list.value[0])    
    
#     if ((abs(lat - np.float32(9.346658)) < 1e-8) and (abs(lon - np.float32(42.075893) < 1e-8))):
#         print(f'Matching record found: {tf_r}')
#         matched_record = tf_r
    

# First TFRecord path
sample_tfrecord = TFRECORDS_DIR + 'ethiopia_00.tfrecord'

# Get feature array of first record
feature_set = helper.create_single_feature_set(filename=sample_tfrecord)
helper.get_feature_array(feature_set=feature_set)

{'SW_IR1': ('float_list', (65025,)),
 'consumption': ('float_list', (1,)),
 'NIR': ('float_list', (65025,)),
 'SW_IR2': ('float_list', (65025,)),
 'BLUE': ('float_list', (65025,)),
 'lon': ('float_list', (1,)),
 'LON': ('float_list', (65025,)),
 'LAT': ('float_list', (65025,)),
 'system:index': ('bytes_list', (1,)),
 'RED': ('float_list', (65025,)),
 'density': ('float_list', (1,)),
 'DELTA_TEMP': ('float_list', (65025,)),
 'CO': ('float_list', (65025,)),
 'TEMP': ('float_list', (65025,)),
 'lat': ('float_list', (1,)),
 'VIIRS': ('float_list', (65025,)),
 'country': ('bytes_list', (1,)),
 'GREEN': ('float_list', (65025,))}

In [101]:
def view_single_record_bands(feature_set):
    lat = np.float32(feature_set['lat'].float_list.value[0])
    lon = np.float32(feature_set['lon'].float_list.value[0])
    country = feature_set['country'].bytes_list.value[0].decode()
    
    band_images = []
    for band_id, band_name in enumerate(BANDS):
        band = np.array(feature_set[band_name].float_list.value, dtype=np.float32).reshape(255, 255)
        band_images.append(band)
    band_images = np.stack(band_images, axis=2)
    print(np.shape(band_images))
    
    print(f'{country}: {lat:4f}, {lon:4f}')
    helper.visualize_bands(img=band_images, band_names=BANDS, bands_per_row=3)
    
    
view_single_record_bands(feature_set=feature_set)

(255, 255, 10)
ethiopia: 14.029943, 38.541546


In [103]:
REQUIRED_KEYS = [
    'BLUE', 'GREEN', 'LAT', 'LON', 'VIIRS', 'DELTA_TEMP', 'CO', 'NIR', 'RED', 'SW_IR1', 'SW_IR2', 'TEMP', 'lat',
    'lon', 'consumption', 'density', 'country'
]

def validate_record(feature_set, record_name):
    # Check for missing bands
    missing_req_keys = [key for key in REQUIRED_KEYS if key not in feature_set]
    if len(missing_req_keys) > 0:
        print(f'Record {record_name} missing required keys: {missing_req_keys}')
        return False
    
    lon = np.float32(feature_set['lon'].float_list.value[0])
    lat = np.float32(feature_set['lat'].float_list.value[0])
    
    # Check middle pixel coordinates
    img_lon = np.float32(np.asarray(feature_set['LON'].float_list.value).reshape(255, 255)[127, 127])
    img_lat = np.float32(np.asarray(feature_set['LAT'].float_list.value).reshape(255, 255)[127, 127])

    if abs(lon - img_lon) > 1e-3:
        print(f'Record {record_name} contains mismatch: "lon"={lon}, "LON"={img_lon}')
        return False
    if abs(lat - img_lat) > 1e-3:
        print(f'Record {record_name} contains invalid lat. "lat"={lat}, "LAT"={img_lat}')
        return False

def process_tfrecords(tfrecord_path, export_dir):
    # Incorrect sorting, solely for visual purposes
    tfrecords = sorted([f for f in os.listdir(tfrecord_path) if not f.startswith('.')])
    
    num_good_records = 0
    for tf_i, tf_r in enumerate(tfrecords):
        filename = tfrecord_path + tf_r
        dataset = tf.data.TFRecordDataset(filenames=[filename])
        
        # Splits each TFRecord file into seperate record files
        for record_i, record in enumerate(iter(dataset)):
            example = tf.train.Example.FromString(record.numpy())
            feature_set = example.features.feature
            
            # Change record name as needed
            record_name = f'ethiopia_{tf_i}_{record_i}'
            is_good_record = validate_record(feature_set=feature_set, record_name=record_name)
            
            if is_good_record != False:    
                # Seralize contents of processed record and write
                os.makedirs(export_dir, exist_ok=True)
                export_path = export_dir + f'/{record_name}' + '.tfrecord'
                
                with tf.io.TFRecordWriter(export_path) as writer:
                    writer.write(example.SerializeToString())
                num_good_records += 1
            
            if (record_i + 1) % 25 == 0:
                print(f'Finished validating {record_i + 1} records')
        
        print(f'Finished validating TFRecord {tf_i + 1}')


In [104]:
process_tfrecords(tfrecord_path=TFRECORDS_DIR, export_dir=EXPORT_DIR)

Finished validating 25 records
Finished validating TFRecord 1
Finished validating 25 records
Finished validating TFRecord 2
Finished validating 25 records
Finished validating TFRecord 3
Finished validating 25 records
Finished validating TFRecord 4
Finished validating 25 records
Finished validating TFRecord 5
Finished validating 25 records
Finished validating TFRecord 6
Finished validating 25 records
Finished validating TFRecord 7
Finished validating 25 records
Finished validating TFRecord 8
Finished validating 25 records
Finished validating TFRecord 9
Finished validating 25 records
Finished validating TFRecord 10
Finished validating 25 records
Finished validating TFRecord 11
Finished validating 25 records
Finished validating TFRecord 12
Finished validating 25 records
Finished validating TFRecord 13
Finished validating 25 records
Finished validating TFRecord 14
Finished validating 25 records
Finished validating TFRecord 15
Finished validating 25 records
Finished validating TFRecord 16
F

Finished validating 25 records
Finished validating TFRecord 131
Finished validating 25 records
Finished validating TFRecord 132
Finished validating 25 records
Finished validating TFRecord 133
Finished validating 25 records
Finished validating TFRecord 134
Finished validating 25 records
Finished validating TFRecord 135
Finished validating 25 records
Finished validating TFRecord 136
Finished validating 25 records
Finished validating TFRecord 137
Finished validating 25 records
Finished validating TFRecord 138
Finished validating 25 records
Finished validating TFRecord 139
Finished validating 25 records
Finished validating TFRecord 140
Finished validating 25 records
Finished validating TFRecord 141
Finished validating 25 records
Finished validating TFRecord 142
Finished validating 25 records
Finished validating TFRecord 143
Finished validating 25 records
Finished validating TFRecord 144
Finished validating 25 records
Finished validating TFRecord 145
Finished validating 25 records
Finished 

Finished validating 25 records
Finished validating TFRecord 260
Finished validating 25 records
Finished validating TFRecord 261
